In [17]:
import cv2
import numpy as np
from PIL import Image
import os
import json
from datetime import datetime

In [18]:
print("OpenCV version:", cv2.__version__)
print("NumPy version:", np.__version__)
print("PIL (Pillow) available")
print("All libraries imported successfully!")

OpenCV version: 4.11.0
NumPy version: 1.26.4
PIL (Pillow) available
All libraries imported successfully!


In [19]:
if not os.path.exists("data"):
    os.makedirs("data")
    print("Created 'data' directory for storing face images")

if not os.path.exists("screenshots"):
    os.makedirs("screenshots")
    print("Created 'screenshots' directory")

In [20]:
def create_users_config():
    users_config = "users.json"

    default_users = {
        "1": "Kalhara",
        "2": "User2",
        "3": "User3"
    }

    if os.path.exists(users_config):
        with open(users_config, 'r') as f:
            users = json.load(f)
        print("Loaded existing users configuration:")
    else:
        users = default_users
        with open(users_config, 'w') as f:
            json.dump(users, f, indent=2)
        print("Created new users configuration:")

    for user_id, name in users.items():
        print(f"    ID: {user_id} - Name: {name}")
    
    return users

In [21]:
def add_user(user_id, name):
    users_config = "users.json"
    if os.path.exists(users_config):
        with open(users_config, 'r') as f:
            users = json.load(f)
    else:
        users = {}
    users[str(user_id)] = name
    with open(users_config, 'w') as f:
        json.dump(users, f, indent=2)
    
    print(f" Added user: {name} (ID: {user_id})")
    return users
        

In [22]:
users = create_users_config()

Loaded existing users configuration:
    ID: 1 - Name: Kalhara
    ID: 2 - Name: User2
    ID: 3 - Name: User3


In [23]:
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [24]:
def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_classifier.detectMultiScale(
        gray, 
        scaleFactor=1.3,    
        minNeighbors=5,     
        minSize=(30, 30)   
    )
    if len(faces) == 0:
        return None
    if len(faces) > 1:
        faces = sorted(faces, key=lambda x: x[2] * x[3], reverse=True)

    x, y, w, h = faces[0]
    cropped_face = img[y:y+h, x:x+w]
    
    return cropped_face
    
    
    

In [25]:
def test_face_detection():
    print("Testing face detection...")
    print(" Press 'q' to quit, 's' to save a detected face")
    
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Cannot open camera")
        return
    
    saved_count = 0 
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Cannot read frame")
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"Face {len(faces)}", (x, y - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.imshow("Face Detection Test", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('s') and len(faces) > 0:
            face = detect_face(frame)
            if face is not None:
                saved_count += 1
                filename = f"test_face_{saved_count}.jpg"
                cv2.imwrite(filename, face)
                print(f" Saved face as {filename}")
    cap.release()
    cv2.destroyAllWindows()
    print(f" Face detection test completed. Saved {saved_count} faces.")

In [26]:
def collect_face_data(user_id=1, num_samples=200):
    print(f" Starting data collection for User ID: {user_id}")
    print(f" Target samples: {num_samples}")
    print(" Instructions:")
    print("   - Look directly at the camera")
    print("   - Move your head slightly for different angles")
    print("   - Ensure good lighting")
    print("   - Press 'q' to quit early")


    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print(" Error: Cannot open camera")
        return False
    
    img_count = 0
    
    print("\n Data collection started!")

    while True:
        ret, frame = cap.read()
        if not ret:
            print(" Error: Cannot read frame")
            break
        display_frame = frame.copy()
        cv2.putText(display_frame, f"User ID: {user_id}", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(display_frame, f"Samples: {img_count}/{num_samples}", (10, 60), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(display_frame, "Press 'q' to quit", (10, 90), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.imshow("Data Collection - Original", display_frame)
        cropped_face = detect_face(frame)
        
        if cropped_face is not None:
            img_count += 1
            face_resized = cv2.resize(cropped_face, (200, 200))
            face_gray = cv2.cvtColor(face_resized, cv2.COLOR_BGR2GRAY)
            filename = f"data/user.{user_id}.{img_count}.jpg"
            cv2.imwrite(filename, face_gray)
            display_face = face_gray.copy()
            cv2.putText(display_face, f"Sample: {img_count}", (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            cv2.imshow("Data Collection - Processed Face", display_face)

            if img_count % 10 == 0:
                print(f" Collected {img_count}/{num_samples} samples")
            if img_count >= num_samples:
                break

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()
    
    print(f"\n Data collection completed!")
    print(f" Total samples collected: {img_count}")
    print(f" Files saved in 'data' directory with prefix 'user.{user_id}'")
    
    return True

In [27]:
def check_collected_data():
    print("TRAINING DATA SUMMARY")
    print("=" * 40)

    if not os.path.exists("data"):
        print("No data directory found")
        return
    
    files = [f for f in os.listdir("data") if f.endswith('.jpg')]
    
    if len(files) == 0:
        print("No training images found")
        return

    user_counts = {}
    for filename in files:
        try:
            user_id = filename.split('.')[1]
            user_counts[user_id] = user_counts.get(user_id, 0) + 1
        except:
            continue

    users_config = "users.json"
    if os.path.exists(users_config):
        with open(users_config, 'r') as f:
            users = json.load(f)
    else:
        users = {}

    total_samples = 0
    for user_id, count in user_counts.items():
        name = users.get(user_id, f"User {user_id}")
        print(f" {name} (ID: {user_id}): {count} samples")
        total_samples += count
    
    print(f"\n Total samples: {total_samples}")
    print(f" Users with data: {len(user_counts)}")

In [28]:
def train_face_recognizer():
    print("Starting model training...")
    if not os.path.exists("data"):
        print("Error: 'data' directory not found")
        print("Collect training data first using collect_face_data()")
        return False

    image_files = [f for f in os.listdir("data") if f.endswith('.jpg')]
    
    if len(image_files) == 0:
        print("Error: No training images found")
        print("Collect training data first using collect_face_data()")
        return False
    print(f" Found {len(image_files)} training images")
    faces = []
    labels = []
    print(" Processing training images...")
    for i, image_file in enumerate(image_files):
        try:
            image_path = os.path.join("data", image_file)
            img = Image.open(image_path).convert('L')
            face_np = np.array(img, 'uint8')
            user_id = int(image_file.split('.')[1])
            faces.append(face_np)
            labels.append(user_id)
            if (i + 1) % 50 == 0:
                print(f"   Processed {i + 1}/{len(image_files)} images")
        except Exception as e:
            print(f"  Error processing {image_file}: {e}")
            continue
    if len(faces) == 0:
        print("Error: No valid training images found")
        return False
    faces = np.array(faces)
    labels = np.array(labels)
    print(f" Successfully processed {len(faces)} images")
    print(f" Training data for {len(set(labels))} different users")
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.train(faces, labels)
    model_path = "face_recognizer_model.xml"
    recognizer.write(model_path)
    print(f" Model training completed!")
    print(f" Model saved as: {model_path}")
    print(f"\n TRAINING SUMMARY:")
    print(f"   Total images: {len(faces)}")
    print(f"   Unique users: {len(set(labels))}")
    print(f"   Model file: {model_path}")
    user_counts = {}
    for label in labels:
        user_counts[label] = user_counts.get(label, 0) + 1
    print("\n Samples per user:")
    for user_id, count in sorted(user_counts.items()):
        print(f"   User {user_id}: {count} samples")
        
    return True
        
    

In [29]:
def load_trained_model():
    model_path = "face_recognizer_model.xml"
    if not os.path.exists(model_path):
        print(f" Error: Model file '{model_path}' not found")
        print(" Train the model first using train_face_recognizer()")
        return None
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read(model_path)
    print(f" Model loaded successfully from {model_path}")
    return recognizer

In [30]:
def validate_model():
    model_path = "face_recognizer_model.xml"
    if not os.path.exists(model_path):
        print(" No trained model found")
        return False
    model_size = os.path.getsize(model_path) / 1024 
    model_time = datetime.fromtimestamp(os.path.getmtime(model_path))
    print(" MODEL INFORMATION:")
    print(f"   File: {model_path}")
    print(f"   Size: {model_size:.1f} KB")
    print(f"   Created: {model_time.strftime('%Y-%m-%d %H:%M:%S')}")
    recognizer = load_trained_model()
    if recognizer is not None:
        print(" Model validation successful")
        return True
    else:
        print(" Model validation failed")
        return False

In [31]:
def load_users_config():
    users_config = "users.json"
    
    if os.path.exists(users_config):
        with open(users_config, 'r') as f:
            return json.load(f)
    else:
        return {"1": "Kalhara", "2": "Unknown User"}

In [33]:
def recognize_faces(confidence_threshold=75):
    print(f" Starting face recognition...")
    print(f" Confidence threshold: {confidence_threshold}%")
    print(" Controls:")
    print("   - Press 'q' to quit")
    print("   - Press 's' to save screenshot")
    print("   - Press 'c' to change confidence threshold")

    recognizer = load_trained_model()
    if recognizer is None:
        return False

    users = load_users_config()

    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print(" Error: Cannot open camera")
        return False
    
    screenshot_count = 0
    recognition_count = 0
    
    print("\n Face recognition started!")

    while True:
        ret, frame = cap.read()
        if not ret:
            print(" Error: Cannot read frame")
            break
        display_frame = frame.copy()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(
            gray, 
            scaleFactor=1.1,
            minNeighbors=10,
            minSize=(50, 50)
        )
        
        for (x, y, w, h) in faces:
            face_roi = gray[y:y + h, x:x + w]
            user_id, confidence_score = recognizer.predict(face_roi)
            confidence = int(100 * (1 - confidence_score / 300))
            if confidence > confidence_threshold:
                user_name = users.get(str(user_id), f"User {user_id}")
                color = (0, 255, 0)  
                label = f"{user_name} ({confidence}%)"
                recognition_count += 1
            else:
                color = (0, 0, 255) 
                label = f"UNKNOWN ({confidence}%)"
            cv2.rectangle(display_frame, (x, y), (x + w, y + h), color, 2)
            (label_width, label_height), _ = cv2.getTextSize(
                label, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)
            cv2.rectangle(display_frame, (x, y - 35), 
                         (x + label_width, y), color, -1)
            cv2.putText(display_frame, label, (x, y - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        info_y = 30
        cv2.putText(display_frame, f"Faces: {len(faces)}", (10, info_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        info_y += 30
        cv2.putText(display_frame, f"Confidence: {confidence_threshold}%", (10, info_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        info_y += 30
        cv2.putText(display_frame, "q:quit s:screenshot c:change confidence", 
                   (10, info_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
        cv2.imshow("Face Recognition System", display_frame)
        
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            break
        
        elif key == ord('s'):
            screenshot_count += 1
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"screenshots/recognition_{timestamp}_{screenshot_count}.jpg"
            cv2.imwrite(filename, display_frame)
            print(f" Screenshot saved: {filename}")
        
        elif key == ord('c'):
            print(f"\n Current confidence threshold: {confidence_threshold}%")
            try:
                new_threshold = int(input("Enter new threshold (50-95): "))
                if 50 <= new_threshold <= 95:
                    confidence_threshold = new_threshold
                    print(f"Confidence threshold changed to: {confidence_threshold}%")
                else:
                    print("  Invalid range. Using previous value.")
            except:
                print("  Invalid input. Using previous value.")
    cap.release()
    cv2.destroyAllWindows()

    print(f"\n Face recognition session completed!")
    print(f" Total recognitions: {recognition_count}")
    print(f" Screenshots saved: {screenshot_count}")
    
    return True

In [34]:
def test_single_image(image_path):
    print(f"  Testing recognition on: {image_path}")
    recognizer = load_trained_model()
    if recognizer is None:
        return False
    users = load_users_config()
    if not os.path.exists(image_path):
        print(f" Error: Image file not found: {image_path}")
        return False
    img = cv2.imread(image_path)
    if img is None:
        print(f" Error: Cannot load image: {image_path}")
        return False
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if len(faces) == 0:
        print(" No faces detected in the image")
        return False
    
    print(f" Found {len(faces)} face(s)")
    
    for i, (x, y, w, h) in enumerate(faces):
        face_roi = gray[y:y + h, x:x + w]
        user_id, confidence_score = recognizer.predict(face_roi)
        confidence = int(100 * (1 - confidence_score / 300))
        user_name = users.get(str(user_id), f"User {user_id}")
        print(f"  Face {i+1}: {user_name} (Confidence: {confidence}%)")
        color = (0, 255, 0) if confidence > 75 else (0, 0, 255)
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, f"{user_name} ({confidence}%)", (x, y - 10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
    cv2.imshow("Single Image Recognition", img)
    print("👀 Press any key to close the image window")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return True

In [ ]:
def display_menu():
    print("\n" + "="*50)
    print(" FACE RECOGNITION SYSTEM")
    print("="*50)
    print("1.Add New User")
    print("2.Collect Training Data")
    print("3.Train Model")
    print("4.Start Face Recognition")
    print("5.Test Single Image")
    print("6.View System Status")
    print("7.Test Camera & Face Detection")
    print("8.Run Complete Setup Workflow")
    print("9.Exit")
    print("="*50)
    
def view_system_status():
    print("\n SYSTEM STATUS")
    print("="*40)
    if os.path.exists("data"):
        files = [f for f in os.listdir("data") if f.endswith('.jpg')]
        print(f" Training images: {len(files)}")
        
        user_counts = {}
        for f in files:
            try:
                user_id = f.split('.')[1]
                user_counts[user_id] = user_counts.get(user_id, 0) + 1
            except:
                continue
        
        users = load_users_config()
        for user_id, count in user_counts.items():
            name = users.get(user_id, f"User {user_id}")
            print(f"    {name}: {count} samples")
    else:
        print("No training data found")
    
    if os.path.exists("face_recognizer_model.xml"):
        model_size = os.path.getsize("face_recognizer_model.xml") / 1024
        model_time = datetime.fromtimestamp(os.path.getmtime("face_recognizer_model.xml"))
        print(f" Model: {model_size:.1f} KB")
        print(f" Last trained: {model_time.strftime('%Y-%m-%d %H:%M:%S')}")
    else:
        print(" No trained model found")
    
    users = load_users_config()
    print(f" Registered users: {len(users)}")
    for user_id, name in users.items():
        print(f"   ID {user_id}: {name}")
    
def run_complete_workflow():
    print("\nCOMPLETE SETUP WORKFLOW")
    print("="*50)
    print("This will guide you through the complete setup process:")
    print("1. User management")
    print("2. Data collection") 
    print("3. Model training")
    print("4. Testing recognition")
    print()
    
    input("Press Enter to start...")
    
    print("\nSTEP 1: USER MANAGEMENT")
    print("-" * 30)
    
    users = load_users_config()
    print("Current users:")
    for user_id, name in users.items():
        print(f"   ID {user_id}: {name}")
    
    while True:
        add_user_choice = input("\nAdd a new user? (y/n): ").lower()
        if add_user_choice != 'y':
            break
        
        try:
            new_id = input("Enter user ID (number): ")
            new_name = input("Enter user name: ")
            
            if new_id and new_name:
                add_user(int(new_id), new_name)
                users = load_users_config()
            else:
                print("Please provide both ID and name")
        except ValueError:
            print("Please enter a valid number for user ID")
    
    print("\nSTEP 2: DATA COLLECTION")
    print("-" * 30)
    
    for user_id, name in users.items():
        collect_choice = input(f"\nCollect training data for {name} (ID: {user_id})? (y/n): ").lower()
        
        if collect_choice == 'y':
            try:
                samples = input("Number of samples (default 200): ") or "200"
                samples = int(samples)
                
                print(f"\nGet ready to collect data for {name}")
                print("Position yourself in front of the camera with good lighting")
                input("Press Enter when ready...")
                
                success = collect_face_data(int(user_id), samples)
                if success:
                    print(f"Data collection completed for {name}")
                else:
                    print(f"Data collection failed for {name}")
                    
            except ValueError:
                print("Invalid number of samples")
    
    print("\nSTEP 3: MODEL TRAINING")
    print("-" * 30)
    
    train_choice = input("\nTrain the face recognition model? (y/n): ").lower()
    if train_choice == 'y':
        success = train_face_recognizer()
        if success:
            print("Model training completed successfully!")
        else:
            print("Model training failed")
        
